In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !mkdir -p /content/drive/MyDrive/bdq-mt/bdq-vie-mt
%cd /content/drive/MyDrive/bdq-mt/bdq-vie-mt

/content/drive/MyDrive/bdq-mt/bdq-vie-mt


# Data Preprocessing

## Data Filtering
*   Deleting empty rows;
*   Deleting duplicates;
*   Deleting source-copied rows;
*   Deleting too long Source/Target (ratio 200% and > 200 words);
*   Removing HTML;
*   Segments will remain in the true-case unless lower is True;
*   Shuffling rows

In [ ]:
!python /content/drive/MyDrive/bdq-mt/data-processing/filtering/filter.py /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.vie bdq vie

Dataframe shape (rows, columns): (35750, 2)
--- Rows with Empty Cells Deleted	--> Rows: 35750
--- Duplicates Deleted			--> Rows: 35693
--- Source-Copied Rows Deleted		--> Rows: 35591
--- Too Long Source/Target Deleted	--> Rows: 34975
--- HTML Removed			--> Rows: 34975
--- Rows will remain in true-cased	--> Rows: 34975
--- Rows with Empty Cells Deleted	--> Rows: 34975
--- Rows Shuffled			--> Rows: 34975
--- Source Saved: /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq-filtered.bdq
--- Target Saved: /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.vie-filtered.vie


In [ ]:
!head -n 3 /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq-filtered.bdq && echo "-----" && head -n 3 /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.vie-filtered.vie

Kră Yang gô ăn kơ iĕm pŭn hiôk tơ jơ̆p-jang anih, tơ lăm pơlei dah tơ mir chŭn
Ih yua tơdrong mơsêh pơjing lu kông gei kơjăp,
atau
-----
Chúa sẽ ban phước cho anh chị em ở khắp mọi nơi, trong thành thị cũng như ngoài đồng ruộng
Ngài dùng quyền năng thiết lập núi non,
đứng


## Data Splitting

In [ ]:
!python /content/drive/MyDrive/bdq-mt/data-processing/train_dev_split/tran_dev_test_split.py 2000 2000 /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq-filtered.bdq /content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.vie-filtered.vie

Dataframe shape: (34975, 2)
--- Empty Cells Deleted --> Rows: 34975
--- Wrote Files
Done!
Output files
/content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq-filtered.bdq.train
/content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.vie-filtered.vie.train
/content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq-filtered.bdq.eval
/content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.vie-filtered.vie.eval
/content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq-filtered.bdq.test
/content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.vie-filtered.vie.test


## Tokenization


### Train a Subwording Model


In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm

In [ ]:
train_source_file_tok = "/content/drive/MyDrive/bdq-mt/dataset/bdq-vie/all.vie-bdq.bdq-filtered.bdq.train"
train_target_file_tok = "/content/drive/MyDrive/bdq-mt/bdq-vie-mt/data/all.vie-bdq.vie-filtered.vie.train"

In [ ]:
source_train_value = '--input='+train_source_file_tok+' --model_prefix=subwording/source --vocab_size=14000 --hard_vocab_limit=false --model_type=bpe --split_digits=true'
spm.SentencePieceTrainer.train(source_train_value)
print("Done, training a SentencepPiece model for the Source finished successfully!")

Done, training a SentencepPiece model for the Source finished successfully!


In [ ]:
!python /content/drive/MyDrive/bdq-mt/data-processing/subwording/2-subword.py subwording/source.model data/all.vie-bdq.bdq-filtered.bdq.train

Model: subwording/source.model
Dataset: data/all.vie-bdq.bdq-filtered.bdq.train
Done subwording the file! Output: data/all.vie-bdq.bdq-filtered.bdq.train.subword


In [ ]:
!python /content/drive/MyDrive/bdq-mt/data-processing/subwording/2-subword.py subwording/source.model data/all.vie-bdq.bdq-filtered.bdq.eval

Model: subwording/source.model
Dataset: data/all.vie-bdq.bdq-filtered.bdq.eval
Done subwording the file! Output: data/all.vie-bdq.bdq-filtered.bdq.eval.subword


In [ ]:
target_train_value = '--input='+train_target_file_tok+' --model_prefix=subwording/target --vocab_size=32000 --hard_vocab_limit=false --model_type=bpe --split_digits=true'
spm.SentencePieceTrainer.train(target_train_value)
print("Done, training a SentencepPiece model for the Target finished successfully!")

Done, training a SentencepPiece model for the Target finished successfully!


In [ ]:
!python /content/drive/MyDrive/bdq-mt/data-processing/subwording/2-subword.py subwording/target.model data/all.vie-bdq.vie-filtered.vie.train

Model: subwording/target.model
Dataset: data/all.vie-bdq.vie-filtered.vie.train
Done subwording the file! Output: data/all.vie-bdq.vie-filtered.vie.train.subword


In [ ]:
!python /content/drive/MyDrive/bdq-mt/data-processing/subwording/2-subword.py subwording/target.model data/all.vie-bdq.vie-filtered.vie.eval

Model: subwording/target.model
Dataset: data/all.vie-bdq.vie-filtered.vie.eval
Done subwording the file! Output: data/all.vie-bdq.vie-filtered.vie.eval.subword


In [ ]:
!head -n 3 data/all.vie-bdq.bdq-filtered.bdq.train.subword && echo "-----" && head -n 3 data/all.vie-bdq.vie-filtered.vie.train.subword

▁Kră ▁Yang ▁gô ▁ăn ▁kơ ▁iĕm ▁pŭn ▁hiôk ▁tơ ▁jơ̆p - jang ▁anih , ▁tơ ▁lăm ▁pơlei ▁dah ▁tơ ▁mir ▁chŭn
▁Ih ▁yua ▁tơdrong ▁mơsêh ▁pơjing ▁lu ▁kông ▁gei ▁kơjăp ,
▁atau
-----
▁Chúa ▁sẽ ▁ban ▁phước ▁cho ▁anh ▁chị ▁em ▁ở ▁khắp ▁mọi ▁nơi , ▁trong ▁thành ▁thị ▁cũng ▁như ▁ngoài ▁đồng ▁ruộng
▁Ngài ▁dùng ▁quyền ▁năng ▁thiết ▁lập ▁núi ▁non ,
▁đứng


### Pretrained tokenization for Vietnamese Using VNCoreNLP word segmenter and PhoBert Tokenizer


In [ ]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4307 sha256=e6d4568f3c6f73d401653aecacf80aae40a103cc97bd263bb314cf1a80ee0cc5
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='/content/drive/MyDrive/bdq-vie-mt/vncorenlp')

In [ ]:
# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/drive/MyDrive/bdq-vie-mt/vncorenlp')

In [ ]:
text = "Công tác phòng chống tội phạm về kinh tế, ma túy, môi trường được tăng cường triển khai, thực hiện đạt kết quả"

output = rdrsegmenter.word_segment(text)
print(output)

['Công_tác phòng_chống tội_phạm về kinh_tế , ma_tuý , môi_trường được tăng_cường triển_khai , thực_hiện đạt kết_quả']


# Training

## Training Configuration

In [ ]:
!pip3 install OpenNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 29.0 MB/s eta 0:00:00


In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: data/all.vie-bdq.bdq-filtered.bdq.train.subword
        path_tgt: data/all.vie-bdq.vie-filtered.vie.train.subword
        transforms: [filtertoolong]
    valid:
        path_src: data/all.vie-bdq.bdq-filtered.bdq.eval.subword
        path_tgt: data/all.vie-bdq.vie-filtered.vie.eval.subword
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 14000
tgt_vocab_size: 32000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: subwording/source.model
tgt_subword_model: subwording/target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.bdqvie

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 3000

# Default: 10000 - Run validation after n steps
valid_steps: 1000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!cat config.yaml

# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: data/all.vie-bdq.bdq-filtered.bdq.train.subword
        path_tgt: data/all.vie-bdq.vie-filtered.vie.train.subword
        transforms: [filtertoolong]
    valid:
        path_src: data/all.vie-bdq.bdq-filtered.bdq.eval.subword
        path_tgt: data/all.vie-bdq.vie-filtered.vie.eval.subword
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 14000
tgt_vocab_size: 32000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: subwording/source.model
tgt_subword_model: subwording/target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.bdqvie

#

## Build Vocab

In [ ]:
!nproc --all

2


In [ ]:
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

2024-01-28 17:43:28.557853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 17:43:28.557908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 17:43:28.559214: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 17:43:28.566513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 17:43:29.619611: W tensorflow/compiler/tf2

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c866bbec-eb3f-7495-f250-d505dad12e43)


In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14999.0625 out of: 15102.0625


## Training

In [ ]:
!onmt_train -config config.yaml

2024-01-28 17:43:40.811513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 17:43:40.811559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 17:43:40.812785: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 17:43:40.819864: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 17:43:41.895616: W tensorflow/compiler/tf2

# Translation

In [ ]:
# !onmt_translate -model models/model.bdqvie_step_3000.pt -src data/paralle.vie-bdq.bdq-filtered.bdq.subword.test -output data/paralle.vie.translated -gpu 0 -min_length 1

2024-01-28 17:06:28.304060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 17:06:28.304121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 17:06:28.305523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 17:06:28.313047: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 17:06:29.423848: W tensorflow/compiler/tf2

In [ ]:
# !tail -n 5 data/paralle.vie-bdq.bdq-filtered.bdq.subword.test

▁' Nĕ ▁kơ ▁achăng ▁bôl ▁buăl ▁ih ▁dah ▁buăl ▁juăt ▁' bă ▁ih ,
▁puôn
▁chơpuih ▁( hnam )
▁' Boi ▁quy ▁hoach ▁kŭm ▁trương ▁đơ̆ng ▁khu ▁vưk ▁lơ̆m ▁apung ▁teh ▁ngoai ▁thanh ▁uĕi ▁lơ̆m ▁kơbră ▁phương ▁Nhơn ▁Phŭ ▁jê̆ ▁Trương ▁Cao ▁đăng ▁' Binh ▁Đinh ▁weng ▁Trương ▁Đai ▁hŏk ▁Kuang ▁Trung
▁Baasa ▁lôch ▁năm ▁oei ▁hơdai ▁hăm ▁lu ▁' bok ▁kơdră ▁sư , ▁păng ▁đe ▁' bŭ ▁sư ▁tơ ▁Pơlei ▁Tirsa


In [ ]:
# !tail -n 5 data/paralle.vie.translated

▁Đừng ▁từ ▁bỏ ▁bạn ▁con ▁hay ▁bạn ▁của ▁cha ▁con ;
▁bốn
▁quét ▁( nhà )
▁Theo ▁quy ▁hoạch ▁cụm ▁trường ▁của ▁khu ▁vực ▁tại ▁khu ▁đất ▁ngoại ▁thành ▁thuộc ▁phường ▁Nhơn ▁Phú ▁gần ▁Trường ▁Cao ▁đẳng ▁Bình ▁Định ▁và ▁Trường ▁Đại ▁học ▁Quang ▁Trung .
▁Ba - ê - sa ▁an ▁giấc ▁với ▁các ▁tổ ▁phụ ▁mình ▁và ▁được ▁chôn ▁tại ▁Tiệt - sa


In [ ]:
# Desubword the translation file
# !python /content/drive/MyDrive/bdq-mt/data-processing/subwording/3-desubword.py subwording/target.model data/paralle.vie.translated

Done desubwording! Output: data/paralle.vie.translated.desubword


In [ ]:
# Desubword the target file (reference) of the test dataset
# !python3 /content/drive/MyDrive/bdq-mt/data-processing/subwording/3-desubword.py subwording/target.model data/paralle.vie-bdq.vie-filtered.vie.subword.test

Done desubwording! Output: data/paralle.vie-bdq.vie-filtered.vie.subword.test.desubword


In [ ]:
!pip3 install ctranslate2

In [ ]:
!ct2-opennmt-py-converter --model_path models/model.bdqvie_step_3000.pt --output_dir models/bavi_ctranslate2 --quantization int8

In [ ]:
!python /content/drive/MyDrive/bdq-mt/evaluation/translate.py data/all.vie-bdq.bdq-filtered.bdq.test subwording/source.model subwording/target.model models/bavi_ctranslate2

Done


In [ ]:
!tail -n 10 data/all.vie-bdq.bdq-filtered.bdq.test.translated

Cây điều chịu trách nhiệm được các phương châm đa dạng và khắc phục hậu quả tang đất với lượng mưa năm đủ và có một mùa khô ráo.
Thấy Giô-sép cha mình suy nghiệm, anh con số người đó không được ơn trước mặt Giô-sép, và không còn nói ra nữa
Sết sống tất cả được chín trăm mười hai năm, rồi qua đời
Có ai nói với Đức Chúa Trời:
học tập những chính sách có liên quan đến hoạt động của Ban Quản lý Di tích.
màu tím
Lời của CHÚA phán như vầy: “ Có một lần, mỗi năm một lần như những công bố những công việc của Đức Chúa Trời đã làm cho ngươi
Vua Đa-vít nói với toàn thể hội chúng: “Giờ đây, con trai con gái của CHÚA, Đức Chúa Trời có thể chọn, gọi ta, ta không thể nào ghi nhớ việc này
Thời gian thực hiện: từ tháng 9 năm 2018 đến hết tháng 12 năm 2020 (20), học kỳ học 90 hàng năm học theo).
sốt


In [ ]:
!tail -n 10 data/all.vie-bdq.vie-filtered.vie.test

Cây điều chịu được những điều kiện khí hậu đa dạng và khắc nghiệt. Khí hậu nhiệt đới với lượng mưa hàng năm đầy đủ và có một mùa khô rõ rệt là những điều kiện tối thích để cây điều phát triển tốt. Độ cao nơi trồng điều so với mặt biển càng lớn thì cây sinh trưởng càng chậm, năng suất càng giảm.
Các anh Giô-sép ganh tỵ vì thấy cha thiên vị, nên họ không còn dịu ngọt với Giô-sép nữa
Sết sống tất cả chín trăm mười hai năm, rồi qua đời
Có ai từng thưa với Đức Chúa Trời:
Hoặc bổ sung các chính sách liên quan đến hoạt động của Ban Quản lý Di tích.
bầm tím
Vì Chúa đã phán cùng tôi như vầy: “Trong vòng một năm, như các năm của một người làm thuê, tất cả vinh quang của Kê-đa sẽ chấm dứt
Vua Đa-vít nói với toàn hội chúng: “Sa-lô-môn, con trai ta, người mà Đức Chúa Trời đã chọn, hãy còn trẻ, thiếu kinh nghiệm, còn công việc thì lớn lao vì đền này không phải xây cho người phàm nhưng cho CHÚA, là Đức Chúa Trời
Thời gian thực hiện: Từ tháng 9/2018 đến hết tháng 12/2020 (05 học kỳ, tương đương 90 tuầ

# Evaluation

In [ ]:
!pip3 install sacrebleu

In [ ]:
!python3 /content/drive/MyDrive/bdq-mt/evaluation/eval-metrics.py   data/all.vie-bdq.vie-filtered.vie.test data/all.vie-bdq.bdq-filtered.bdq.test.translated

BLEU:  49.61
CHRF: 59.51
TER: 48.49
